# *TP1 - DEEP LEARNING*

# Partie 1 : _Formalisation mathématique_

Pour appliquer un réseau de neurones à un problème de machine learning (en apprentissage supervisé),
on a besoin de 4 choses :

- Un jeu de données annoté et un problème associé ;
- Une architecture de réseau (à adapter aux données) ;
- Une fonction de coût que l’on cherche à minimiser (à adapter au problème) ;
- Un algorithme d’apprentissage pour résoudre le problème d’optimisation consistant à minimiser
cette fonction de coût.

On s'intéresse ici à un problème de classification en apprentissage supervisé. On a donc un jeu de données constitué d'un ensemble de $N$ couples $ (x^{(i)}, y^{(i)}) $, où $ i \in \{1, ..., N\} $, $ x^{(i)} \in \mathbb{R}^{n_x} $ est un vecteur de _features_ à partir duquel on veut prédire la vérité terrain $ y^{(i)} \in \{0, 1\}^{n_y} $ vérifiant $ \|y^{(i)}\| = 1 $ (_one hot encoding_). 

Ce jeu de données est généralement découpé en plusieurs ensembles : **train**, **test** et parfois **val**.

### Q1. A quoi servent les ensembles d’apprentissage, de validation et de test ?


Train : ajuster les paramètres du modèle.
Validation (val) : choisir l’architecture / hyperparamètres et détecter le surapprentissage.
Test : évaluer la performance finale sur des données jamais vues.


### Q2. Quelle est l’influence du nombre $N$ d’exemples ?

Plus N est grand, meilleure est l’estimation des paramètres, moins de variance et meilleure généralisation (si qualité des labels stable).
Trop peu d’exemples → surapprentissage ; données bruitées → performance limitée même si N grand

#### Q3. Pourquoi est-il important d’ajouter des fonctions d’activation entre des transformations lineaires ?

Pour introduire la non-linéarité permettant d’approximer des fonctions complexes ; sans elles, le réseau reste une transformation linéaire unique quelle que soit la profondeur.

#### Q4. Quelles sont les tailles $ n_{x}, n_{h}, n_{y} $ sur la figure 1  ? En pratique, comment ces tailles sont-elles choisies ?

n_x : dimension des features d’entrée.
n_h : nombre d’unités dans la couche cachée (taille latente / capacité).
n_y : nombre de classes (dimension de la sortie one-hot).
Choix pratique : n_x et n_y sont dictés par les données ; n_h choisi par validation croisée / contraintes de calcul en équilibrant sous- et surcapacité.

#### Q5. Que représentent les vecteurs $ y $ et $ \hat{y} $ ? Quelle est la différence entre ces deux quantités ?

y : vérité terrain (one-hot) indiquant la classe réelle.
ŷ : prédiction probabiliste du modèle (vecteur de scores / probabilités).
Différence : y est binaire/indicateur, ŷ est continu (probabilités) ; objectif : rapprocher ŷ de y.

#### Q6. Pourquoi utiliser une fonction $ SoftMax $ en sortie ?

Convertit les scores en probabilités positives qui somment à 1, permettant interprétation probabiliste et usage direct avec la cross-entropy pour classification multi-classe.

#### Q7. Écrire les équations mathématiques permettant d'effectuer la passe forward du réseau de neurones, produisant successivement $ \tilde{h}, \hat{h}, \tilde{y}, $ et $ \hat{y} $ à partir de $ x $.

\tilde{h} = W^{(1)} x + b^{(1)}
\hat{h} = f(\tilde{h}) (activation, ex. ReLU, tanh)
\tilde{y} = W^{(2)} \hat{h} + b^{(2)}
\hat{y} = softmax(\tilde{y})

#### Q8. Pendant l’apprentissage, on cherche à minimiser la fonction de coût. Pour l’entropie croisée et l’erreur quadratique, comment les $ \hat{y}_i $ doivent-ils varier pour faire diminuer la loss ?

Cross-entropy : augmenter la probabilité ŷ_c de la vraie classe c (tendre vers 1) et diminuer les autres.
MSE (avec one-hot) : rapprocher chaque composante ŷ_i de y_i (pour la vraie classe augmenter vers 1, les autres vers 0).

#### Q9. En quoi ces fonctions sont-elles plus adaptées aux problèmes de classification ou de régression ?

Classification multi-classe : softmax + cross-entropy.
Classification binaire : sigmoid + binary cross-entropy.
Régression : sorties continues + MSE ou autres pertes adaptées (MAE, Huber).

#### Q10. Quels semblent etre les avantages et inconvenients des diverses variantes de descente de gradient entre les versions classique, stochastique sur mini-batch et stochastique online ? Laquelle semble la plus raisonnable a utiliser dans le cas general ?

Batch (toute la dataset) : gradients exacts, couteux en mémoire et temps pour grands jeux de données.
Online (un échantillon) : très bruité, converge vite mais instable.
Mini-batch : compromis pratique (stabilité, efficacité GPU, bruit utile pour évasion minima locaux).
Recommandation générale : mini-batch SGD (taille choisie selon mémoire / variance)

#### Q11. Quelle est l’influence du learning rate $ η $ sur l’apprentissage ?

Trop grand → divergence / oscillations ; trop petit → convergence lente, risque de rester bloqué.
Utiliser schedule/adaptatif (decay, Adam, etc.) et régler par validation.

#### Q12. Comparer la complexité (en fonction du nombre de couches du réseau)  du calcul des gradients de la loss par rapport aux paramètres, en utilisant l'approche naïve et l'algorithme de l’algorithme de backprop .

Naïve : recalculer dérivées séparément pour chaque paramètre entraîne coût multiplicatif (croissance exponentielle/linéaire en couches selon implémentation), très cher.
Backprop : calcule gradients pour tous les paramètres en temps proportionnel à un petit multiple de la passe forward (complexité linéaire en nombre d’opérations du réseau), donc beaucoup plus efficace.

#### Q13. Quel critère doit respecter l’architecture du réseau pour permettre la backpropagation ?

Toutes les opérations doivent être différentiables (au moins presque partout) et le graphe computationnel doit être acyclique ; les paramètres doivent influencer la sortie via ce graphe.

#### Q14. La fonction $ SoftMax $ et la loss de cross-entropy sont souvent utilisées ensemble et leur gradient est très simple. Montrez que la loss se simplifie en :

Pour logits \tilde{y} et vraie classe c, la loss = -log softmax_c(\tilde{y}) = -\tilde{y}_c + log(∑_j exp(\tilde{y}_j)).
C’est la forme usuelle utilisée pour la stabilité numérique.

#### Q15. Écrire le gradient de la loss (cross-entropy) par rapport à la sortie intermédiaire $\tilde{y}$ :

∂L/∂\tilde{y}_i = \hat{y}_i - y_i (pour chaque composante i).

In [ ]:
# Vérification: softmax + cross-entropy et gradient analytique
import numpy as np
def softmax(logits):
    e = np.exp(logits - np.max(logits))
    return e / e.sum()
def cross_entropy_loss(logits, y_onehot):
    s = softmax(logits)
    return -np.log(s[y_onehot==1])
# Exemple
logits = np.array([2.0, 1.0, 0.1])
y_onehot = np.array([1, 0, 0])
s = softmax(logits)
loss = cross_entropy_loss(logits, y_onehot)
analytical_grad = s - y_onehot
# gradient numérique via différences finies
eps = 1e-6
num_grad = np.zeros_like(logits)
for i in range(len(logits)):
    l = logits.copy()
    l[i] += eps
    lp = cross_entropy_loss(l, y_onehot)
    l[i] -= 2*eps
    lm = cross_entropy_loss(l, y_onehot)
    num_grad[i] = (lp - lm) / (2*eps)
print('logits:', logits)
print('softmax:', np.round(s,6))
print('loss:', float(loss))
print('analytical_grad:', np.round(analytical_grad,6))
print('numeric_grad:', np.round(num_grad,6))
print('max abs diff:', np.max(np.abs(analytical_grad - num_grad)))